# 1. Importing necessary libraries and packages

In [1]:
from scx.optimize import Model
import pandas as pd

# 2. Data Collection and Preprocessing

In [2]:
# Load Demand, Facility Data, Transportation Cost from xlxs file (DataInput_Scen3.xlsx)
df_outdist=pd.read_excel('DataInput.xlsx',sheet_name='OutDist',skiprows=1,index_col=0) # Outbound Distance Table (DCs - Retailers)
df_out_transcost=pd.read_excel('DataInput.xlsx',sheet_name='OutTransCost',skiprows=1,index_col=0) # Outbound Transportation Cost Table
df_demand=pd.read_excel('DataInput.xlsx',sheet_name='Demand',skiprows=1) # Retailer's Demand Table
df_dccost=pd.read_excel('DataInput.xlsx',sheet_name='CWCost',skiprows=1, index_col=0) # DC's Data Table (including Variable Cost, Fixed Cost)
df_indist=pd.read_excel('DataInput.xlsx',sheet_name='InDist',skiprows=1, index_col=0) # Inbound Distance Table (Plants - DC)
df_pcost_cap=pd.read_excel('DataInput.xlsx',sheet_name='PCost_Cap',skiprows=1, index_col=0) # Plant's Data Talbe (including Variable Cost, Capacity)

In [3]:
# List of Plant, DC, Retail load from input table above
list_plant=df_indist.columns.to_list()
list_dc=df_indist.index.to_list()
list_r=df_outdist.columns.to_list()

In [4]:
in_transcost=0.06 # inbound transportation cost
M=9999 # a big number use for setting up Linking Constraint 

# List of Inbound Transportation flow, including all relevant information loaded from data in above Tables
inbound_trans=[]
for plant in list_plant:
    for dc in list_dc:
        dict={}
        dict['origin']=plant
        dict['destination']=dc
        dict['dist']=df_indist.loc[dc,plant]
        dict['trans_cost']=in_transcost
        dict['p_var_cost']=df_pcost_cap.loc[plant,'Variable costs']
        dict['dc_var_cost']=df_dccost.loc[dc,'Variable costs']
        inbound_trans.append(dict)

# List of Outbound Transportation flow, including all relevant information loaded from data in above Tables
outbound_trans=[]
for dc in list_dc:
    for r in list_r:
        dict={}
        dict['origin']=dc
        dict['destination']=r
        dict['dist']=df_outdist.loc[dc,r]
        dict['trans_cost']=df_out_transcost.loc[dc,r]
        if dict['dist']<=75: 
            dict['<=80km?']= 1
        else : 
            dict['<=80km?']=0
        outbound_trans.append(dict)
        
# List of Retailers and its Demand
rs_demand=[]
for r in list_r:
    dict={}
    dict['name']=r
    dict['demand']=df_demand.loc[0,r]
    rs_demand.append(dict)

# List of Plants and its Supply (Capacity)
plants_supply=[]
for plant in list_plant:
    dict={}
    dict['name']=plant
    dict['capacity']=df_pcost_cap.loc[plant,'Capacity']
    plants_supply.append(dict)

# List of DCs and its Fixed Cost
dcs_cost=[]
for dc in list_dc:
    dict={}
    dict['name']=dc
    dict['fixed_cost']=df_dccost.loc[dc,'Fixed costs']
    dcs_cost.append(dict)

In [5]:
# check inbound transportation flow
inbound_trans[0]

{'origin': 'Plant1',
 'destination': 'DC1',
 'dist': 860,
 'trans_cost': 0.06,
 'p_var_cost': 8,
 'dc_var_cost': 8}

In [6]:
# Check Outbound Transportation Flow
outbound_trans[0]

{'origin': 'DC1',
 'destination': 'R1',
 'dist': 89,
 'trans_cost': 1.9,
 '<=80km?': 0}

# 3. Formulated MILPs Model

## 3.1 Define Variables

In [7]:
# 'amt' variable is an "Amount" of product flow through arcs
for inbound in inbound_trans:
    inbound['amt']=Model.variable(name=f"{inbound['origin']}_to_{inbound['destination']}_amt",lowBound=0,cat='Integer')
for outbound in outbound_trans:
    outbound['amt']=Model.variable(name=f"{outbound['origin']}_to_{outbound['destination']}_amt",lowBound=0,cat='Integer')
# 'open?' is binary variable indicating whether the DC is open (1) or not (0)
for dc in dcs_cost:
    dc['open?']=Model.variable(name=f"{dc['name']}_open?", cat='Binary')

In [8]:
# recheck Intbound Transportation Flow (after adding "amt" variable)
inbound_trans[0]

{'origin': 'Plant1',
 'destination': 'DC1',
 'dist': 860,
 'trans_cost': 0.06,
 'p_var_cost': 8,
 'dc_var_cost': 8,
 'amt': Plant1_to_DC1_amt}

In [9]:
# recheck Outbound Transportation Flow (after adding "amt" variable)
outbound_trans[0]

{'origin': 'DC1',
 'destination': 'R1',
 'dist': 89,
 'trans_cost': 1.9,
 '<=80km?': 0,
 'amt': DC1_to_R1_amt}

In [10]:
# recheck DCs Status (after adding "open?" variable)
dcs_cost[0]

{'name': 'DC1', 'fixed_cost': 20000, 'open?': DC1_open?}

## 3.2 Model Initialization

In [11]:
my_model=Model('EcoTreat',sense='Minimize')

### Add Objective

In [12]:
my_model.add_objective(fn=sum(inbound['amt']*inbound['p_var_cost'] for inbound in inbound_trans) # Plant variable operating costs
                       + sum(inbound['amt']*inbound['dist']*inbound['trans_cost'] for inbound in inbound_trans) # Inbound transportation costs (Plants - DCs)
                       + sum(dc['fixed_cost']*dc['open?'] for dc in dcs_cost) # Distribution Center fixed costs
                       + sum(inbound['amt']*inbound['dc_var_cost'] for inbound in inbound_trans) # Distribution Center variable operating costs
                       + sum(outbound['amt']*outbound['dist']*outbound['trans_cost'] for outbound in outbound_trans) # Outbound transportation costs (DCs - Rs)
                      )

### Add Constraint

#### Demand

In [13]:
# Demand Constraint: Total product served to a specific Rs must meet its demand
for demand in rs_demand:
    my_model.add_constraint(name=f"{demand['name']}_demand", 
                            fn=sum(outbound['amt'] for outbound in outbound_trans if outbound['destination']==demand['name'])==demand['demand'])


#### Capacity 

In [14]:
# Supply Constraints: Total product produced by a specific plant must not exceed its capacity
for supply in plants_supply:
    my_model.add_constraint(name=f"{supply['name']}_supply", 
                            fn=sum(inbound['amt'] for inbound in inbound_trans if inbound['origin']==supply['name'])<=supply['capacity'])

#### Flow Balance

In [15]:
# Flow Balance Constraints: Total input product of each DC must equal total output
for dc in list_dc:
    my_model.add_constraint(name=f"{dc}_flow_balance",
                           fn=sum(inbound['amt'] for inbound in inbound_trans if inbound['destination']==dc)
                            ==sum(outbound['amt'] for outbound in outbound_trans if outbound['origin']==dc))

#### Linking 

In [16]:
# Linking Constraints: The binary variable for DC selection must reflect the actual selection status
for dc in dcs_cost:
    my_model.add_constraint(name=f"{dc['name']}_open?",
                           fn=(sum(outbound['amt'] for outbound in outbound_trans if outbound['origin']==dc['name'])-M*dc['open?'])<=0)

#### Number of DCs

In [17]:
# Open 2 DCs
my_model.add_constraint(name='Min DC open',
                       fn=sum(dc['open?'] for dc in dcs_cost)>=0)
my_model.add_constraint(name='Max DC open',
                       fn=sum(dc['open?'] for dc in dcs_cost)<=2)

In [18]:
# Force DC4 is one of 2 which opened
for dc in dcs_cost:
    if dc['name']=='DC4':
        my_model.add_constraint(name='DC4 is opening',
                                fn= (dc['open?']==1))
    else:
        None

#### Level Of Service

In [19]:
# 80% of Retailers demand to be within 75 kilometers of the DC
my_model.add_constraint(name='60% of Retailer demand to be within 80 km of the DC',
                       fn=(sum(outbound['<=80km?']*outbound['amt'] for outbound in outbound_trans)/sum(demand['demand'] for demand in rs_demand))>=0.8
                       )

In [20]:
# Average weighted distance from DC-R <= 65 kilometers
my_model.add_constraint(name='Average weighted distance from DC-R <=80',
                       fn=(sum(outbound['dist']*outbound['amt'] for outbound in outbound_trans)/sum(demand['demand'] for demand in rs_demand))<=65
                       )

## Solve and get output

In [21]:
my_model.solve()
my_model.show_formulation()

EcoTreat:
MINIMIZE
20000*DC1_open? + 162.0*DC1_to_R10_amt + 158.1*DC1_to_R11_amt + 123.2*DC1_to_R12_amt + 88.0*DC1_to_R13_amt + 94.8*DC1_to_R14_amt + 136.5*DC1_to_R15_amt + 169.1*DC1_to_R1_amt + 148.2*DC1_to_R2_amt + 69.0*DC1_to_R3_amt + 114.0*DC1_to_R4_amt + 87.6*DC1_to_R6_amt + 96.80000000000001*DC1_to_R7_amt + 80.30000000000001*DC1_to_R8_amt + 111.8*DC1_to_R9_amt + 127.5*DC1_to__R5_amt + 9999999*DC2_open? + 112.2*DC2_to_R10_amt + 111.0*DC2_to_R11_amt + 61.199999999999996*DC2_to_R12_amt + 51.0*DC2_to_R13_amt + 90.0*DC2_to_R14_amt + 120.0*DC2_to_R15_amt + 90.0*DC2_to_R1_amt + 100.8*DC2_to_R2_amt + 75.6*DC2_to_R3_amt + 109.8*DC2_to_R4_amt + 58.0*DC2_to_R6_amt + 142.4*DC2_to_R7_amt + 128.0*DC2_to_R8_amt + 71.5*DC2_to_R9_amt + 178.6*DC2_to__R5_amt + 16000*DC3_open? + 117.8*DC3_to_R10_amt + 89.10000000000001*DC3_to_R11_amt + 108.80000000000001*DC3_to_R12_amt + 90.0*DC3_to_R13_amt + 123.0*DC3_to_R14_amt + 133.0*DC3_to_R15_amt + 137.7*DC3_to_R1_amt + 114.8*DC3_to_R2_amt + 113.60000000000001

In [22]:
my_model.show_outputs()

{'objective': 1427480.28,
 'status': 'Optimal',
 'variables': {'DC1_open?': 0.0,
               'DC1_to_R10_amt': 0.0,
               'DC1_to_R11_amt': 0.0,
               'DC1_to_R12_amt': 0.0,
               'DC1_to_R13_amt': 0.0,
               'DC1_to_R14_amt': 0.0,
               'DC1_to_R15_amt': 0.0,
               'DC1_to_R1_amt': 0.0,
               'DC1_to_R2_amt': 0.0,
               'DC1_to_R3_amt': 0.0,
               'DC1_to_R4_amt': 0.0,
               'DC1_to_R6_amt': 0.0,
               'DC1_to_R7_amt': 0.0,
               'DC1_to_R8_amt': 0.0,
               'DC1_to_R9_amt': 0.0,
               'DC1_to__R5_amt': 0.0,
               'DC2_open?': 0.0,
               'DC2_to_R10_amt': 0.0,
               'DC2_to_R11_amt': 0.0,
               'DC2_to_R12_amt': 0.0,
               'DC2_to_R13_amt': 0.0,
               'DC2_to_R14_amt': 0.0,
               'DC2_to_R15_amt': 0.0,
               'DC2_to_R1_amt': 0.0,
               'DC2_to_R2_amt': 0.0,
               'DC2_to